# 4. Building a Simple Agent in Python

Welcome to this tutorial on building a simple agent in Python! In this notebook, we will focus on creating a basic agent that can communicate with a **Large Language Model (LLM)** to perform tasks autonomously. We will build upon the concepts you learned in the previous notebook, such as prompts and LLM interaction, and now put those into practice to create a functional agent.

By the end of this tutorial, you will know how to configure a simple agent, interact with an LLM, and process the responses it provides.

## What is an Agent?

An **Agent** is an autonomous system that leverages **Large Language Models (LLMs)** to perform tasks by understanding, reasoning, planning, and executing actions with minimal human intervention. AI agents are designed to break down complex problems into manageable steps, utilizing tools, accessing memory, and adapting their behavior based on the provided context.

At its core, an agent is structured to:

1. **Receive a Task**: The agent takes input from the user, such as a question or command.
2. **Plan a Solution**: The agent decomposes the problem, chooses appropriate tools, and reasons through possible solutions.
3. **Execute the Plan**: It performs actions, such as retrieving information, using tools, or generating responses based on the devised plan.
4. **Deliver Results**: Finally, it presents the solution or output in a structured, actionable format.

![agenticvnonagentic](images/agentic-vs-non-agentic.png)

For more in-depth analysis on LLM-based autonomous agents, refer to this survey: [A Survey on Large Language Model based Autonomous Agents](https://arxiv.org/pdf/2308.11432)

### Key Components of an AI Agent:

- **Profiling Module (Agent Core)**: This is the agent's decision-making hub. It defines the role and goals of the agent (e.g., financial analyst, teacher), selects appropriate tools, and coordinates task execution. The agent's "profile" helps determine its behavior and interaction style based on its role.
  
- **Memory Module**: The agent uses memory to track past interactions and experiences. Short-term memory stores context-relevant information (e.g., current session details), while long-term memory retains important information over time, which the agent can refer back to when needed.

- **Tools Module**: External resources (e.g., APIs, databases) that the agent can call upon to complete tasks, like retrieving real-time data, performing calculations, or interacting with other systems.

- **Planning Module**: This module allows the agent to break down complex tasks into smaller, manageable sub-tasks. By planning step-by-step, the agent can tackle intricate queries and tasks with greater efficiency and precision.

![agentmodules](images/agent_modules_small.png)

### Example:

Consider an agent designed to assist with financial analysis. When asked, "What are the three key takeaways from the Q2 earnings call for FY 2023?", the agent processes the query in a few steps:
1. It first identifies the relevant sections of the earnings call.
2. Next, it analyzes the key points related to business or technology developments.
3. Lastly, the agent summarizes and presents the findings clearly to the user.

This example highlights how AI agents harness the reasoning capabilities of an LLM, combined with external tools and memory, to generate well-structured and detailed responses to complex tasks.

## 1. Setting up the Environment

Before we can communicate with the LLM, let’s install any required libraries and ensure our environment is ready.

In [ ]:
%pip install requests jsonschema tenacity

These packages are used for:

- **requests:** Making HTTP requests to interact with models.
- **jsonschema:** Validating the structure of the agent's output.
- **tenacity:** Handling retries in case of errors when communicating with the model.

### Datetime Function

We'll create is a simple function to get the current time. This is important because our agent might need to timestamp certain actions or events. Let's write a function that returns the current date and time in UTC format:

In [ ]:
from datetime import datetime, timezone


def get_current_utc_datetime():
    now_utc = datetime.now(timezone.utc)
    return now_utc.strftime("%Y-%m-%d %H:%M:%S.%f UTC")[:-3]


# Example usage:
print("Current UTC datetime:", get_current_utc_datetime())

## 2. Configuring a Simple Model

In this section, we configure the machine learning model that our agent will use to process tasks. The `ModelService` class manages the interaction with the model (in this case, "llama3.1:8b-instruct-fp16"), allowing the agent to handle tasks such as listing VMs and retrieving details.

### Model Configuration

We initialize the `ModelService` with a specific model configuration, including parameters such as model endpoint, temperature (for controlling randomness), and others. This step enables our agent to perform model-based tasks using the provided configuration.

In [ ]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:latest")

## 3. System Prompt for the Simple Agent

In this cell, we define the **System Prompt** for our agent. The system prompt helps set the behavior of the agent when interacting with the **Large Language Model (LLM)**. This particular prompt instructs the LLM to act as a helpful assistant, focused on answering questions clearly and concisely.

### Key Aspects of the System Prompt:
1. **Assistant Role:** The agent is instructed to provide useful, clear, and concise responses to user queries.
2. **Guidelines:**
   - **Clear Responses:** The agent is expected to give direct and simple answers to the user's questions.
   - **Avoid Unnecessary Details:** It avoids overloading the user with too much information—responses should be brief and relevant.
   - **Polite Tone:** The agent will always maintain a friendly and helpful tone in its responses.
   - **Answer Format:** Responses are expected to be returned in simple text, with no special formatting unless specifically requested by the user.
   
### Example Interaction:
This system prompt also includes an example to illustrate the expected behavior of the agent:
- If the user asks, "What is Python used for?", the agent will respond with something like:
  - "Python is a versatile programming language used for web development, data analysis, automation, and more."

This **System Prompt** ensures that the agent remains consistent in how it responds, providing useful and easy-to-understand answers in all cases.


In [ ]:
SYS_PROMPT = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutoff Knowledge Date: December 2023
Current Date: {datetime}

You are a helpful assistant. Your job is to answer questions clearly and concisely. Make sure your responses are easy to understand and provide useful information.

---

### Example Interaction:

User: "What is Python used for?"

Assistant: "Python is a versatile programming language used for web development, data analysis, automation, and more."

---

### Key Points:
- **Be Direct:** Answer only what is asked, without extra information.
- **Be Polite:** Maintain a friendly and helpful tone.
<|eot_id|>
"""

## 4.Agent Class for Interacting with the LLM

In this cell, we define the `Agent` class, which serves as the core component responsible for interacting with the **Large Language Model (LLM)**. This class is designed to handle the interaction between user input and the LLM, process the model's responses, and maintain an internal state.

### Key Components:

1. **Initialization (`__init__`):**
   - The `Agent` class is initialized with:
     - `state`: A dictionary to track the agent's current state.
     - `role`: The defined role of the agent (e.g., "assistant").
     - `ollama_service`: A service interface that handles communication with the LLM (in this case, represented by `ModelService`).

2. **State Management (`update_state`):**
   - The `update_state` method allows the agent to update its internal state based on key-value pairs. This helps the agent track progress or store important data.
   - If the agent tries to update a state key that doesn't exist, it will warn the user.

3. **Model Interaction (`invoke_model`):**
   - This is the core method that interacts with the LLM. It prepares the input payload by combining the **system prompt** and **user prompt**.
   - The payload is then sent to the model service (`ollama_service`), which communicates with the LLM, retrieves the response, and processes it.
   - The final processed response from the model is returned for further use.

4. **Main Task Execution (`work`):**
   - The `work` method is the primary interface for executing tasks based on user input.
   - It formats the **system prompt** (using a predefined prompt template) and combines it with the **user prompt** (the user's specific question or request).
   - The `work` method then invokes the model and returns the final response from the LLM.
   - This method can be easily extended for more complex tasks.

### Example Workflow:
- The agent receives a **user request**.
- It prepares the system and user prompts and sends them to the model.
- The LLM processes the input and returns a response.
- The agent processes the LLM’s response and returns the final output.

This structure makes it easy to build on top of the agent, allowing you to handle more complex interactions, manage state efficiently, and extend the agent’s behavior over time.


In [ ]:
from typing import Dict, Any


class Agent:

    def __init__(self, state: Dict[str, Any], role: str, ollama_service: ModelService):
        """
        Initialize the Agent with a state, role, and model configuration.
        """
        self.state = state
        self.role = role
        self.ollama_service = ollama_service

    def update_state(self, key: str, value: Any):
        """
        Update the state of the agent. Warn if the key doesn't exist.
        """
        if key in self.state:
            self.state[key] = value
        else:
            print(f"Warning: Attempting to update a non-existing state key '{key}'.")

    def invoke_model(self, sys_prompt: str, user_prompt: str):
        """
        Prepare the payload, send the request to the model, and process the response.
        """
        # Prepare the payload
        payload = self.ollama_service.prepare_payload(
            user_prompt,
            sys_prompt,
        )

        # Invoke the model and get the response
        response_json = self.ollama_service.request_model_generate(
            payload,
        )

        # Process the model's response
        response_content = self.ollama_service.process_model_response(response_json)

        # Return the processed response
        return response_content

    def work(
        self,
        user_request: str,
        sys_prompt: str = SYS_PROMPT,
    ) -> str:
        """
        Execute a simple task based on the user's request.
        """
        # Define a simple system prompt
        formatted_sys_prompt = sys_prompt.format(datetime=get_current_utc_datetime())
        user_prompt = f"""<|start_header_id|>user<|end_header_id|>\n\n{user_request}<|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>"""

        # Invoke the model with the user's request
        response = self.invoke_model(
            sys_prompt=formatted_sys_prompt, user_prompt=user_prompt
        )

        # Return the processed response
        return response

### Running the Agent

Finally, let's demonstrate how to run the agent with a simple example. We'll use the agent class we've just implemented to process a task based on user input.

In [ ]:
# Initialize the agent with an empty state and a role
agent_state = {"response": ""}
agent_role = "Helper Agent"
agent = Agent(state=agent_state, role=agent_role, ollama_service=ollama_service)

# Execute a task
user_input = "Tell me everything you know about Tesla."
response = agent.work(user_request=user_input)
print("Agent's response:", response)

## 5. Conclusion

In this notebook, we’ve built a **simple AI-powered agent** that interacts with a **Large Language Model (LLM)** to answer user queries. Let’s recap what we've accomplished:

- **Defined the agent's behavior** through a **System Prompt**, instructing the LLM on how to respond to questions.
- **Created the `Agent` class** to manage interactions between the user and the LLM.
- **Implemented state management** to allow the agent to track progress or other necessary information.
- **Configured a model service** to handle communication between the agent and the LLM, enabling it to retrieve and process responses.
- **Executed the agent** with a user query, receiving a helpful response from the LLM.
